In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

from pymongo import MongoClient
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
client = MongoClient('localhost:27017')
db = client.Movie_User_db

In [3]:
def scrapeMovie(url):
	# get the imdbID
	imdbId = url.split('/')[-2][2:]

	# request to main webpage
	response = requests.get(url)
	source = response.text
	soup = BeautifulSoup(source, 'lxml')

	title = soup.find('div', class_='title_wrapper')
	name = title.h1.text[:-8]
	year = title.h1.a.text

	time = title.find('div', class_='subtext').time.text.strip()

	genres = title.find('div', class_='subtext').find_all('a')
	genres.pop()
	for i in range(len(genres)):
		genres[i] = genres[i].text

	poster = soup.find('div', class_='poster').a.img['src']

	rating = soup.find('div', class_='ratingValue').span.text

	# request to plot webpage
	response = requests.get(url+'plotsummary')
	source = response.text
	soup = BeautifulSoup(source, 'lxml')

	summary = soup.find('ul', id='plot-summaries-content').li.p.text

	# Create and return dictionary
	json = {'imdbId':imdbId, 'title':name, 'releaseYear':year, 'thumbnail':poster, 'imdbRating':rating, 'synopsis':summary, 'duration':time, 'genres':genres}

	return json

if __name__=="__main__":
    # print(scrapeMovie("https://www.imdb.com/title/tt8108164/"))
	# scrapeMovie("https://www.imdb.com/title/tt0468569/")
	print("https://www.imdb.com/title/tt0468569/".split('/')[-2][2:])




0468569


In [9]:
scrapeMovie("https://www.imdb.com/title/tt0468569/")

{'imdbId': '0468569',
 'title': 'The Dark Knight',
 'releaseYear': '2008',
 'thumbnail': 'https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_UX182_CR0,0,182,268_AL__QL50.jpg',
 'imdbRating': '9.0',
 'synopsis': 'When the menace known as the Joker emerges from his mysterious past, he wreaks havoc and chaos on the people of Gotham. The Dark Knight must accept one of the greatest psychological and physical tests of his ability to fight injustice.',
 'duration': '2h 32min',
 'genres': ['Action', 'Crime', 'Drama']}

In [4]:
links_df = pd.read_csv("ml-latest-small/links.csv", dtype=str)

In [5]:
links_df[:2]

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844


In [6]:
movies_df = pd.read_csv("moviesWithIndex.csv")

In [10]:
def fillDb():
    for document in tqdm(db.movies.find()):
        movieId = (document["location"].split(".")[0])
        movie = links_df.loc[links_df["movieId"] == movieId]
        a=(movie["imdbId"])
        a=(list(a)[0])
        link = "https://www.imdb.com/title/tt{}/".format(a)
        try:
            scr = scrapeMovie(link)
            synopsis = scr["synopsis"]
            duration = scr["duration"]
            db.synopsis.insert_one({"location": document["location"], "synopsis": synopsis, "duration": duration})
        except:
            db.synopsis.insert_one({"location": document["location"], "synopsis": ""})
    #     print(synopsis)

In [11]:
fillDb()

334it [12:46,  2.30s/it]
